In [8]:
import datajoint as dj
import numpy as np
import time
from pykdtree.kdtree import KDTree

In [2]:
schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')
dj.config["display.limit"] = 30

Connecting celiib@10.28.0.34:3306


In [3]:
pinky.LeftoverCompartmentFinal()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio
3,648518346341371119,0.35
3,648518346349386137,0.35
3,648518346349470171,0.35
3,648518346349471156,0.35
3,648518346349471500,0.35
3,648518346349471562,0.35
3,648518346349471565,0.35
3,648518346349471910,0.35
3,648518346349472574,0.35
3,648518346349472601,0.35


In [36]:
pinky.CoarseLabelOrphan & "segment_id=648518346341352891"

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices Corrected vertex labels,triangles Same triangle labels as pinky.ProofreadLabelOrphan
3,648518346341352891,0.35,computer_Auto,celiib,2019-06-23 10:05:42,=BLOB=,=BLOB=


In [42]:
key_source = ((dj.U("segmentation","segment_id") & pinky.CoarseLabelFinal.proj()) 
     + (dj.U("segmentation","segment_id") & pinky.CoarseLabelOrphan.proj()))
key_source

segmentation segmentation id,segment_id segment id unique within each Segmentation
3,648518346341371119
3,648518346349386137
3,648518346349470171
3,648518346349471156
3,648518346349471500
3,648518346349471562
3,648518346349471565
3,648518346349471910
3,648518346349472574
3,648518346349472601


In [37]:


@schema
class UndecimatedNeuronLabels(dj.Computed):
    definition = """
    # Undecimated Mesh Coarse and Overlayed Labels
    -> pinky.Mesh
    ---
    coarse_vertices_labels             : longblob # vertex labels for the base compartments                   
    coarse_triangles_labels            : longblob # traingle labels for the base compartments
    overlay_vertices_labels             : longblob # vertex labels for the spine compartments overlayed over the base compartments                  
    overlay_triangles_labels            : longblob # triangles labels for the spine compartments overlayed over the base compartments
    """
    
    key_source = ((dj.U("segmentation","segment_id") & pinky.CoarseLabelFinal.proj()) 
     + (dj.U("segmentation","segment_id") & pinky.CoarseLabelOrphan.proj()))

    
    def make(self, key):
        print("\n\n------------- Working on " + str(key["segment_id"]) +"----------------------------------------------")
        #figure out if from the orphan list or excitatory list
        full_time = time.time()
        
        search_key = dict(key,decimation=0.35)
        new_key = key
        
        table = ""
        if len(pinky.CoarseLabelFinal & key) > 0:
            print("classified as Excitatory")
            table="Excitatory"
            dec_vert_labels,dec_tri_labels = (pinky.OverlayedSpineLabel & search_key).fetch1("vertices","triangles")
            #get the decimated mesh
            dec_mesh_table = pinky.PymeshfixDecimatedExcitatoryStitchedMesh & search_key
            dec_vertices, dec_triangles = dec_mesh_table.fetch1("vertices","triangles")


            #make sure that the labels match up:
            print((len(dec_vert_labels),len(dec_vertices)))
            #len(dec_tri_labels),len(dec_triangles)
            
        elif len(pinky.CoarseLabelOrphan & key) > 0:
            print("classified as Orphan")
            table="Orphan"
            dec_vert_labels,dec_tri_labels = (pinky.OverlayedSpineLabelOrphan & search_key).fetch1("vertices","triangles")
            #get the decimated mesh
            dec_mesh_table = pinky.Decimation35OrphanStitched & search_key
            dec_vertices, dec_triangles = dec_mesh_table.fetch1("vertices","triangles")


            #make sure that the labels match up:
            print((len(dec_vert_labels),len(dec_vertices)))
            #len(dec_tri_labels),len(dec_triangles)
        else:
            raise Exception("Segment id was not in Excitatory or Orphan list")
        
        #based on what table it is, get the decimated and label arrays
        
        #get the undecimated mesh
        undec_mesh_table = pinky.Mesh & new_key
        undec_vertices, undec_triangles = undec_mesh_table.fetch1("vertices","triangles")
        print(len(undec_triangles))
        
        from pykdtree.kdtree import KDTree
        start_time = time.time()
        dec_KDTree = KDTree(dec_vertices)
        distances, nearest_nodes = dec_KDTree.query(undec_vertices)
        print(f"Total time for main KDTree creation and queries = {time.time() - start_time}")

        #get the labels for the undecimated mesh
        undecimated_vert_labels = dec_vert_labels[nearest_nodes]

        #get the final labels by combining leftover with regular
        
        if table=="Orphan" or len(pinky.LeftoverCompartmentFinal & key) <= 0:
            print("No leftover meshes to add or just orphan")
            
            final_undec_overlay_verts_labels = np.zeros(len(undecimated_vert_labels))

            error_distance_threshold = 200

            for i in range(0,len(final_undec_overlay_verts_labels)):
                if distances[i] > error_distance_threshold:
                    final_undec_overlay_verts_labels[i] = 10
                else:
                    final_undec_overlay_verts_labels[i] = undecimated_vert_labels[i]
            
            #get the triangle labels for the overlay
            triangle_overlay_labels = final_undec_overlay_verts_labels[undec_triangles[:,0]]

            #get the coarse labels 
            
            if table == "Excitatory":
                dec_vert_labels_coarse,dec_tri_labels_coarse= (pinky.CoarseLabelFinal & search_key).fetch1("vertices","triangles")
                undecimated_vert_labels_coarse = dec_vert_labels_coarse[nearest_nodes]
            else:
                dec_vert_labels_coarse,dec_tri_labels_coarse= (pinky.CoarseLabelOrphan & search_key).fetch1("vertices","triangles")
                undecimated_vert_labels_coarse = dec_vert_labels_coarse[nearest_nodes]
            
            
            #get the final labels by combining leftover with regular
            final_undec_coarse_verts_labels = np.zeros(len(undec_vertices))
            
            for i in range(0,len(final_undec_coarse_verts_labels)):
                if distances[i] > error_distance_threshold:
                    final_undec_coarse_verts_labels[i] = 10
                else:
                    final_undec_coarse_verts_labels[i] = undecimated_vert_labels_coarse[i]
            
            triangle_overlay_labels_coarse = final_undec_coarse_verts_labels[undec_triangles[:,0]]
            #Counter(triangle_overlay_labels_coarse)
            
            #write this to the database
            self.insert1(dict(key,
                        coarse_vertices_labels = final_undec_coarse_verts_labels,                               
                        coarse_triangles_labels= triangle_overlay_labels_coarse,           
                        overlay_vertices_labels = final_undec_overlay_verts_labels,                         
                        overlay_triangles_labels = triangle_overlay_labels ))
            print(f"Total time for mapping: {time.time() - full_time}")

        
        else:
            print("Going to add in the leftover meshes labels as well")
            
            #get the decimated mesh
            dec_vert_labels_leftover,dec_tri_labels_leftover = (pinky.LeftoverOverlayedSpineLabel & search_key).fetch1("vertices","triangles")

            #get the decimated mesh
            dec_mesh_table_leftover = pinky.ExcitatoryLeftoverMeshes & search_key
            dec_vertices_leftover, dec_triangles_leftover = dec_mesh_table_leftover.fetch1("vertices","triangles")


            #make sure that the labels match up:
            print((len(dec_vert_labels_leftover),len(dec_vertices_leftover)))
            #len(dec_tri_labels),len(dec_triangles)

            from pykdtree.kdtree import KDTree
            dec_KDTree_leftover = KDTree(dec_vertices_leftover)

            start_time = time.time()
            distances_leftover, nearest_nodes_leftover = dec_KDTree_leftover.query(undec_vertices)
            print(f"Total time = {time.time() - start_time}")

            #get the labels for the undecimated mesh
            undecimated_vert_labels_leftover = dec_vert_labels_leftover[nearest_nodes_leftover]


            
            
            #get the final labels by combining leftover with regular
            final_undec_overlay_verts_labels = np.zeros(len(undec_vertices))

            error_distance_threshold = 200

            for i in range(0,len(final_undec_overlay_verts_labels)):
                if distances_leftover[i]>error_distance_threshold and distances[i] > error_distance_threshold:
                    final_undec_overlay_verts_labels[i] = 10
                else:
                    if distances[i] < distances_leftover[i]:
                        final_undec_overlay_verts_labels[i] = undecimated_vert_labels[i]
                    else:
                        final_undec_overlay_verts_labels[i] = undecimated_vert_labels_leftover[i]


            triangle_overlay_labels = final_undec_overlay_verts_labels[undec_triangles[:,0]]
            #Counter(triangle_overlay_labels)

            #---------------- Done with getting the overalyed vertices ---------------- ##

            #---------------- Started getting just the coarse labels ---------------- ##

            #get the labels for the undecimated mesh
            #get the decimated mesh
            dec_vert_labels_coarse,dec_tri_labels_coarse= (pinky.CoarseLabelFinal & search_key).fetch1("vertices","triangles")
            undecimated_vert_labels_coarse = dec_vert_labels_coarse[nearest_nodes]


            dec_vert_labels_leftover_coarse,dec_tri_labels_leftover_coarse_coarse = (pinky.LeftoverCoarseLabelFinal & search_key).fetch1("vertices","triangles")
            undecimated_vert_labels_leftover_coarse = dec_vert_labels_leftover_coarse[nearest_nodes_leftover]


            #get the final labels by combining leftover with regular
            final_undec_coarse_verts_labels = np.zeros(len(undec_vertices))

            error_distance_threshold = 200

            for i in range(0,len(final_undec_coarse_verts_labels)):
                if distances_leftover[i]>error_distance_threshold and distances[i] > error_distance_threshold:
                    final_undec_coarse_verts_labels[i] = 10
                else:
                    if distances[i] < distances_leftover[i]:
                        final_undec_coarse_verts_labels[i] = undecimated_vert_labels_coarse[i]
                    else:
                        final_undec_coarse_verts_labels[i] = undecimated_vert_labels_leftover_coarse[i]


            triangle_overlay_labels_coarse = final_undec_coarse_verts_labels[undec_triangles[:,0]]

            
            #write this to the database
            self.insert1(dict(key,
                        coarse_vertices_labels = final_undec_coarse_verts_labels,                               
                        coarse_triangles_labels= triangle_overlay_labels_coarse,           
                        overlay_vertices_labels = final_undec_overlay_verts_labels,                         
                        overlay_triangles_labels = triangle_overlay_labels ))
        
            print(f"Total time for mapping: {time.time() - full_time}")

            



In [40]:
#(schema.jobs & "table_name='__undecimated_neuron_labels'")#.delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [41]:
UndecimatedNeuronLabels()

segmentation segmentation id,segment_id segment id unique within each Segmentation,coarse_vertices_labels vertex labels for the base compartments,coarse_triangles_labels traingle labels for the base compartments,overlay_vertices_labels vertex labels for the spine compartments overlayed over the base compartments,overlay_triangles_labels triangles labels for the spine compartments overlayed over the base compartments
3,648518346341352891,=BLOB=,=BLOB=,=BLOB=,=BLOB=
3,648518346341353058,=BLOB=,=BLOB=,=BLOB=,=BLOB=
3,648518346341353574,=BLOB=,=BLOB=,=BLOB=,=BLOB=
3,648518346341353607,=BLOB=,=BLOB=,=BLOB=,=BLOB=
3,648518346341353788,=BLOB=,=BLOB=,=BLOB=,=BLOB=
3,648518346341354313,=BLOB=,=BLOB=,=BLOB=,=BLOB=
3,648518346341354496,=BLOB=,=BLOB=,=BLOB=,=BLOB=
3,648518346341355048,=BLOB=,=BLOB=,=BLOB=,=BLOB=
3,648518346341355198,=BLOB=,=BLOB=,=BLOB=,=BLOB=
3,648518346341355736,=BLOB=,=BLOB=,=BLOB=,=BLOB=


In [39]:
start = time.time()
UndecimatedNeuronLabels.populate(reserve_jobs=True)
print(time.time() - start)

#648518346341352891



------------- Working on 648518346341352891----------------------------------------------
classified as Orphan
(21316, 21316)
124576
Total time for main KDTree creation and queries = 0.013398885726928711
No leftover meshes to add or just orphan
Total time for mapping: 0.2687950134277344


------------- Working on 648518346341353058----------------------------------------------
classified as Orphan
(38766, 38766)
222198
Total time for main KDTree creation and queries = 0.022555828094482422
No leftover meshes to add or just orphan
Total time for mapping: 0.46547865867614746


------------- Working on 648518346341353574----------------------------------------------
classified as Orphan
(58629, 58629)
336849
Total time for main KDTree creation and queries = 0.031290292739868164
No leftover meshes to add or just orphan
Total time for mapping: 0.5819222927093506


------------- Working on 648518346341353607----------------------------------------------
classified as Orphan
(68095, 68095)
3

OperationalError: (1213, 'Deadlock found when trying to get lock; try restarting transaction')

In [28]:
    key_source = ((dj.U("segmentation","segment_id") & pinky.CoarseLabelFinal.proj()) 
     + (dj.U("segmentation","segment_id") & pinky.CoarseLabelOrphan.proj()))

In [31]:
pinky.CoarseLabelOrphan & "segment_id=648518346341352891"

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices Corrected vertex labels,triangles Same triangle labels as pinky.ProofreadLabelOrphan
3,648518346341352891,0.35,computer_Auto,celiib,2019-06-23 10:05:42,=BLOB=,=BLOB=


In [44]:
pinky.CoarseLabelOrphan()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices Corrected vertex labels,triangles Same triangle labels as pinky.ProofreadLabelOrphan
3,648518346341352891,0.35,computer_Auto,celiib,2019-06-23 10:05:42,=BLOB=,=BLOB=
3,648518346341353058,0.35,computer_Auto,celiib,2019-06-23 12:08:34,=BLOB=,=BLOB=
3,648518346341353574,0.35,computer_Auto,celiib,2019-06-23 13:05:53,=BLOB=,=BLOB=
3,648518346341353607,0.35,computer_Auto,celiib,2019-06-23 13:08:39,=BLOB=,=BLOB=
3,648518346341353788,0.35,computer_Auto,celiib,2019-06-23 13:09:40,=BLOB=,=BLOB=
3,648518346341354313,0.35,computer_Auto,celiib,2019-06-23 16:04:08,=BLOB=,=BLOB=
3,648518346341354496,0.35,computer_Auto,celiib,2019-06-23 16:05:29,=BLOB=,=BLOB=
3,648518346341355048,0.35,computer_Auto,celiib,2019-06-23 16:05:59,=BLOB=,=BLOB=
3,648518346341355198,0.35,computer_Auto,celiib,2019-06-23 16:06:47,=BLOB=,=BLOB=
3,648518346341355736,0.35,computer_Auto,celiib,2019-06-23 16:07:10,=BLOB=,=BLOB=


In [45]:
pinky.CoarseLabelFinal - pinky.LeftoverCompartmentFinal.proj()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices Corrected vertex labels,triangles Same triangle labels as pinky.ProofreadLabel
3,648518346349475436,0.35,computer_Auto,celiib,2019-06-13 13:01:24,=BLOB=,=BLOB=
3,648518346349475529,0.35,computer_Auto,celiib,2019-06-13 13:37:17,=BLOB=,=BLOB=
3,648518346349475534,0.35,computer_Auto,celiib,2019-06-13 13:38:06,=BLOB=,=BLOB=
3,648518346349475573,0.35,computer_Auto,celiib,2019-06-13 14:02:35,=BLOB=,=BLOB=
3,648518346349505813,0.35,computer_Auto,celiib,2019-06-21 21:03:52,=BLOB=,=BLOB=
3,648518346349509347,0.35,computer_Auto,celiib,2019-06-22 10:55:42,=BLOB=,=BLOB=
